In [2]:
#pip install "sagemaker>=2.140.0" "transformers==4.26.1" "datasets[s3]==2.10.1" --upgrade

In [3]:
# %%capture
# import IPython
# !conda install -c conda-forge ipywidgets -y
# IPython.Application.instance().kernel.do_shutdown(True)

In [38]:
import sagemaker
import boto3
sess = sagemaker.Session()
sagemaker_session_bucket = None
if sagemaker_session_bucket is None and sess is not None:
    sagemaker_session_bucket = sess.default_bucket()

role = sagemaker.get_execution_role()
sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)
region = boto3.Session().region_name

In [40]:
from sagemaker.huggingface.model import HuggingFaceModel

# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
   model_data="s3://image-captioning-yurui/model.tar.gz",  # path to your trained SageMaker model
   role=role,                                            # IAM role with permissions to create an endpoint
   transformers_version="4.26",                           # Transformers version used
   pytorch_version="1.13",                                # PyTorch version used
   py_version='py39',                                    # Python version used
)



In [43]:
# deploy model to SageMaker Inference
from sagemaker.deserializers import  JSONDeserializer, StringDeserializer
from sagemaker.serializers import JSONSerializer

predictor = huggingface_model.deploy(
   initial_instance_count=1,
   instance_type="ml.m5.xlarge",
)

----!

In [46]:
# example request: you always need to define "inputs"
data = {
   "inputs": ['https://huggingface.co/datasets/Narsil/image_dummy/raw/main/parrots.png']
}

# request
predictor.predict(data)[0]

'["a pair of parrots standing next to each other"]'

In [37]:
# delete endpoint
predictor.delete_endpoint()

In [3]:
# from sagemaker.huggingface import HuggingFaceModel
# from sagemaker import get_execution_role

hub = {
    'HF_MODEL_ID': 'nlpconnect/vit-gpt2-image-captioning',
    'HF_TASK': 'image-to-text',
}

# model = HuggingFaceModel(
#     env=hub,
#     role=role,
#     transformers_version='4.26',
#     pytorch_version='1.13',
#     py_version='py39',
# )

# predictor = model.deploy(initial_instance_count=1, instance_type='ml.m5.xlarge')


In [24]:
from sagemaker.predictor import Predictor
from sagemaker.huggingface.model import HuggingFacePredictor
predictor = HuggingFacePredictor("huggingface-pytorch-inference-2023-04-28-08-11-51-052")

In [25]:
predictor